In [9]:
# docker

# LangChain notebook 

LangChain is a framework for developing applications powered by language models.

- GitHub: https://github.com/hwchase17/langchain
- Docs: https://python.langchain.com/v0.2/docs/introduction/

### Overview:
- Installation
- LLMs
- Prompt Templates
- Chains
- Agents and Tools
- Memory
- Document Loaders
- Indexes

In [10]:
%pip install langchain langchain_community -q

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install dotenv -q

Note: you may need to restart the kernel to use updated packages.


In [12]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

## LLMs in LangChain

The basic building block of LangChain is a Large Language Model which takes text as input and generates more text.

Suppose we want to generate a company name based on the company description, so we will first initialize an OpenAI wrapper. In this case, since we want the output to be more random, we will intialize our model with high temprature.

The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

temperature value--> how creative we want our model to be

0 ---> temperature it means model is  very safe it is not taking any bets.

1 --> it will take risk it might generate wrong output but it is very creative

In [13]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)

/tmp/ipykernel_3728/166103413.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9)


In [14]:
text="what would be a good company name for a company that make colorful socks?"

In [15]:
print(llm.model_name)   # defaults to gpt 3.5 turbo instruct if none is selected
print(llm.predict(text))

gpt-3.5-turbo-instruct


/tmp/ipykernel_3728/229049954.py:2: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict(text))




"Rainbow Footwear Co."


In [16]:
# equivalent
print(llm(text))

/tmp/ipykernel_3728/4203956707.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm(text))




"Rainbow Feet Co."


In [17]:
# equivalent 
print(llm.invoke(text))



1. Rainbow Socks Co.
2. ColorCraze Socks
3. Happy Feet Co.
4. ChromaSock
5. VibrantSteps
6. Chromatic Sox
7. BrightSocks Co.
8. Kaleidosock Co.
9. ColorPop Socks
10. FunkyFoot Co.


## LangChain and HuggingFace

Hugging Face allows us to use open source models:

In [18]:
%pip install --upgrade --quiet huggingface_hub langchain_huggingface

Note: you may need to restart the kernel to use updated packages.


In [36]:
# let's use a basic language model
from langchain_huggingface.llms.huggingface_endpoint import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-large",
    provider="sambanova",
    max_new_tokens=100,
    do_sample=False,
    huggingfacehub_api_token=hf_token
)

In [37]:
print(llm.invoke("translate English to German: How old are you?"))

ValueError: Task 'text-generation' not supported for provider 'sambanova'. Available tasks: ['conversational', 'feature-extraction']

**Note:** there is a problem with this format in LangChain at the moment, I tried several other methods to use HF but they are not working right now (24/06/2025). 

## Prompt Templates

Currently in the above applications we are writing an entire prompt, if you are creating a user directed application then this is not an ideal case

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In many Large Language Model applications we donot pass the user input directly to the Large Language Model, we add the user input to a large piece of text called prompt template

In [40]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],   # give input variables
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
p = prompt_template_name.format(cuisine="indian")
print(p)

I want to open a restaurant for indian food. Suggest a fancy name for this.


Or we can do:

In [41]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks'

## Chains: Combining LLM & Prompts

Now as we have the  **model**:

```python
llm = OpenAI(temperature=0.9)
```

and the **Prompt Template**:

```python
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")
```

Now using Chains we will link together model and the PromptTemplate.

In LangChain, **chains** are modular sequences of steps where each step involves calling a component such as a language model, a prompt template, a retriever, or a tool. Chains are designed to handle complex workflows by linking together multiple operations, enabling tasks like document summarization, question answering, or agentic reasoning. Instead of manually orchestrating each step, chains allow developers to define an end-to-end logic where inputs are processed through a series of transformations, making the application both *reusable* and *scalable*.

### Single Chains 

The simplest and most common type of Chain is LLMChain, which passes the input first to Prompt Template and then to Large Language Model

[**LLMChain**](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.llm.LLMChain.html#llmchain) is responsible to execute the PromptTemplate, For every PromptTemplate we will specifically have an LLMChain

In [42]:
llm = OpenAI(temperature=0.9)

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")

In [43]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
response= chain.run("colorful socks")
print(response)

/tmp/ipykernel_3728/2715051390.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_3728/2715051390.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response= chain.run("colorful socks")




"Rainbow Threads" or "ChromaSocks" 


Another example:

In [44]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

chain = LLMChain(llm=llm, prompt=prompt_template_name)
response=chain.run("Mexican")
print(response)

 

"El Sabroso Cantina" ("The Delicious Cantina") 


If we want to see how the chain is working internally we can pass the parameter `verbose=True`:

In [45]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
response=chain.run("Mexican")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


"Taco Tres Chic"


### Multiple Chains

What if we want to pass the output of a chain to another chain? Then we should construct a [**Sequential Chain**](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.sequential.SequentialChain.html#sequentialchain).
How can we do that? 

For a 'double' chain, for example, we need to construct two single chains and link them: 

In [46]:
llm = OpenAI(temperature=0.6)

# first prompt template
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

# first chain
name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

# second prompt template
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

# second chain
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [48]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("indian")
print(content)



1. Spicy Chicken Tikka Masala 
2. Lamb Vindaloo 
3. Vegetable Samosas 
4. Tandoori Shrimp 
5. Chana Masala 
6. Butter Chicken 
7. Aloo Gobi 
8. Chicken Biryani 
9. Palak Paneer 
10. Naan Bread 
11. Mango Lassi 
12. Garlic Naan 
13. Vegetable Korma 
14. Chicken Tandoori Wrap 
15. Mango Chutney 
16. Dal Makhani 
17. Malai Kofta 
18. Tandoori Mixed Grill 
19. Saag Chicken 
20. Kulfi (Indian ice cream)


The limit with `SimpleSequentialChain` is that it only shows the last output. To overcome this, we can use `SequentialChain`. For this one we have to specify the parameter `output_key` in our `LLMChain`.

In [52]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [53]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

print(chain({"cuisine": "indian"}))


/tmp/ipykernel_3728/2442697741.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain({"cuisine": "indian"}))


{'cuisine': 'indian', 'restaurant_name': '\n\n"Spice Palace"', 'menu_items': ' \n\n1. Spicy Chicken Tikka Masala\n2. Lamb Vindaloo\n3. Vegetable Biryani\n4. Aloo Gobi (potato and cauliflower curry)\n5. Tandoori Shrimp\n6. Saag Paneer (spinach and paneer cheese curry)\n7. Chana Masala (chickpea curry)\n8. Butter Chicken\n9. Naan bread\n10. Samosas\n11. Papadum (crispy lentil crackers)\n12. Mango Lassi (yogurt drink)\n13. Gulab Jamun (fried milk dumplings in syrup)\n14. Kheer (rice pudding)\n15. Masala Chai (spiced tea)'}


## Tools in LangChain 

**Agents** in LangChain enable dynamic decision-making by allowing a language model to choose actions based on the current context, observe the results, and iteratively continue until a final answer is reached. This loop—**decide, act, observe, repeat**—makes agents particularly well-suited for handling complex or multi-step tasks that can't be solved with a single prompt.

When used effectively, agents can be extremely powerful. To work with agents in LangChain, it's important to understand the following core components:

- **Tool**: A callable function that performs a specific task, such as web search, database lookup, math calculation, or invoking another chain. Tools are how the agent interacts with the outside world.
- **LLM (Language Model)**: The underlying model responsible for interpreting the task, selecting tools, and reasoning through the workflow.
- **Agent**: The controller that uses the LLM to decide which tool to invoke, based on the current state of the task and previous observations.

By combining these components, LangChain agents can reason, access external data, and adapt their behavior in real time.


For example I have to travel from Dubai to Canada, I type this in ChatGPT



---> Give me  two flight options from Dubai to Canada on September 1, 2024 | ChatGPT will not be able to answer because has knowledge till
September 2021



ChatGPT plus has Expedia Plugin, if we enable this plugin it will go to Expedia Plugin and will try to pull information about Flights & it will show the information

#### Tool example: Wikipedia

In [72]:
%pip install --upgrade --quiet  wikipedia

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [62]:
# define llm

llm = OpenAI(temperature=0)

In [75]:
# The tools we'll give the Agent access to. 
tools = load_tools(["wikipedia"])

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,    #https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html#agenttype
    verbose=True
)

# Let's test it out!


agent.run("What was the GDP of US in 2024?")



> Entering new AgentExecutor chain...
 I should always think about what to do
Action: wikipedia
Action Input: GDP of US in 2024
Observation: Page: List of U.S. states and territories by GDP
Summary: This is a list of U.S. states and territories by gross domestic product (GDP). This article presents the 50 U.S. states and the District of Columbia and their nominal GDP at current prices.
The data source for the list is the Bureau of Economic Analysis (BEA) in 2024. The BEA defined GDP by state as "the sum of value added from all industries in the state."
Overall, in the calendar year 2024, the United States' Nominal GDP at Current Prices totaled at $29.184 trillion, as compared to $27.720 trillion in 2023.
The three U.S. states with the highest GDPs were California ($4.103 trillion), Texas ($2.709 trillion), and New York ($2.297 trillion). The three U.S. states with the lowest GDPs were Vermont ($45.7 billion), Wyoming ($53.0 billion), and Alaska ($69.9 billion).
GDP per capita also va

'The GDP of the US in 2024 was $29.184 trillion.'

In [77]:
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,    #https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html#agenttype
    verbose=True
)

agent.run("What was the GDP of US in 2024? Retrieve it using the wikipedia tool i gave you. And what is the ratio between the 20224 GDP and 2023 GDP?")



> Entering new AgentExecutor chain...
 I should use the wikipedia tool to search for the GDP of US in 2024.
Action: wikipedia
Action Input: GDP of US in 2024
Observation: Page: List of U.S. states and territories by GDP
Summary: This is a list of U.S. states and territories by gross domestic product (GDP). This article presents the 50 U.S. states and the District of Columbia and their nominal GDP at current prices.
The data source for the list is the Bureau of Economic Analysis (BEA) in 2024. The BEA defined GDP by state as "the sum of value added from all industries in the state."
Overall, in the calendar year 2024, the United States' Nominal GDP at Current Prices totaled at $29.184 trillion, as compared to $27.720 trillion in 2023.
The three U.S. states with the highest GDPs were California ($4.103 trillion), Texas ($2.709 trillion), and New York ($2.297 trillion). The three U.S. states with the lowest GDPs were Vermont ($45.7 billion), Wyoming ($53.0 billion), and Alaska ($69.9 bi

'The GDP of US in 2024 was $29.184 trillion and the ratio between the 2024 GDP and 2023 GDP was 105.28138528138528%.'

> **Note:** Prompts are very important. For example, without specifying that he should use the wikipedia tool, the agent will try to use Wikipedia and fail. That's why prompt engeneering is important. Also we usually compose a prompt of a hardcoded system prompt + the user prompt, in order to avoid mistakes like these.

In [79]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,    #https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html#agenttype
    verbose=True
)

agent.run("What was the GDP of US in 2024? And what is the ratio between the 20224 GDP and 2023 GDP?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to perform mathematical operations.
Action: Calculator
Action Input: (2024 GDP - 2023 GDP) / 2023 GDP
Observation: Answer: 0.0004943153732081067
Thought: I should use the Wikipedia tool to find the GDP of US in 2024 and 2023.
Action: Wikipedia
Action Input: "GDP of US in 2024" and "GDP of US in 2023"
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use the Calculator tool to perform mathematical operations.
Action: Calculator
Action Input: 2024 GDP and 2023 GDP
Observation: Answer: 4047
Thought: I now know the final answer
Final Answer: The GDP of US in 2024 is 4047 and the ratio between the 2024 GDP and 2023 GDP is 0.0004943153732081067.

> Finished chain.


'The GDP of US in 2024 is 4047 and the ratio between the 2024 GDP and 2023 GDP is 0.0004943153732081067.'

## Memory in LangChain

Chatbot application like ChatGPT, you will notice that it remember past information. Does a chain have memory by its own?

In [80]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)

from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)

 

"El Sabroso Cantina"


In [81]:
name = chain.run("Indian")
print(name)



"Masala Majesty"


Let's access `chain.memory`:

In [83]:
type(chain.memory)

NoneType

Ok, there's nothing there. 

This is because LangChain Chains don't automatically store memory if not told so. We can actually use 3 types of memory inside LagnChain:


### Conversation Buffer Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, 
                 prompt=prompt_template_name, 
                 memory=memory) # pass the memory parameter
name = chain.run("Mexican")
print(name)

/tmp/ipykernel_3728/3208601181.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()




"La Cantina de México"


In [86]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"La Cantina de México"


It's remembering the past chat! Amazing.

But there is an issue with `ConversationBufferMemory()`: it remembers all previous conversations, therefore is very heavy in memory. 

To overcome this problem we can use `ConversationChain()`:

### ConversationChain

In [87]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


/tmp/ipykernel_3728/3006166724.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo = ConversationChain(llm=OpenAI(temperature=0.7))


In [88]:
convo.run("My name is Matteo and I am a researcher at the University of Bologna")

" Hello Matteo, it's nice to meet you. I am an AI created by OpenAI and I am always happy to chat with humans. I have access to a vast amount of information and can provide you with specific details related to your research at the University of Bologna. What specifically are you interested in?"

In [89]:
convo.run("What was my name again?")

' Your name is Matteo, and according to my records, you are a researcher at the University of Bologna. Would you like me to provide more information about your research projects or any specific areas of interest you have?'

In [90]:
print(convo.memory.buffer)

Human: My name is Matteo and I am a researcher at the University of Bologna
AI:  Hello Matteo, it's nice to meet you. I am an AI created by OpenAI and I am always happy to chat with humans. I have access to a vast amount of information and can provide you with specific details related to your research at the University of Bologna. What specifically are you interested in?
Human: What was my name again?
AI:  Your name is Matteo, and according to my records, you are a researcher at the University of Bologna. Would you like me to provide more information about your research projects or any specific areas of interest you have?
